In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import dash_table

In [8]:
import company

COMPANY = company.Company("Visa", 5, isin=["10-K"])

filings = COMPANY.df_filings

display(filings)

# filings = filings[:3]

{'cik': '0001403161', 'name': 'VISA INC.'}


,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,cik,InteractiveURL,ExcelURL,df_statements
80,0001403161-23-000099,2023-11-15,2023-09-30,2023-11-15T16:07:14.000Z,34,10-K,001-33977,231410612,,19718853,1,1,v-20230930.htm,10-K,0001403161,https://www.sec.gov/Archives/edgar/data/000140...,https://www.sec.gov/Archives/edgar/data/000140...,Name \ 0 ...
192,0001403161-22-000081,2022-11-16,2022-09-30,2022-11-16T16:06:43.000Z,34,10-K,001-33977,221395232,,25116725,1,1,v-20220930.htm,10-K,0001403161,https://www.sec.gov/Archives/edgar/data/000140...,https://www.sec.gov/Archives/edgar/data/000140...,Name \ 0 ...
283,0001403161-21-000060,2021-11-18,2021-09-30,2021-11-18T16:06:50.000Z,34,10-K,001-33977,211424468,,25113866,1,1,v-20210930.htm,10-K,0001403161,https://www.sec.gov/Archives/edgar/data/000140...,https://www.sec.gov/Archives/edgar/data/000140...,Name \ 0 ...
375,0001403161-20-000070,2020-11-19,2020-09-30,2020-11-19T16:03:34.000Z,34,10-K,001-33977,201328832,,25094543,1,1,v-20200930.htm,10-K,0001403161,https://www.sec.gov/Archives/edgar/data/000140...,https://www.sec.gov/Archives/edgar/data/000140...,Name \ 0 ...
459,0001403161-19-000050,2019-11-14,2019-09-30,2019-11-14T15:35:00.000Z,34,10-K,001-33977,191219544,,28748946,1,1,v093019form10k.htm,10-K,0001403161,https://www.sec.gov/Archives/edgar/data/000140...,https://www.sec.gov/Archives/edgar/data/000140...,Name \ 0 ...


<h1>Helper Functions</h1>

In [9]:
def merge_lists_unique(lists):
    unique_elements = set()
    result = []

    for sublist in lists:
        for element in sublist:
            if element not in unique_elements:
                unique_elements.add(element)
                result.append(element)

    return result

# Example usage:
lists = [
['NET INCOME', 'Foreign-currency translation adjustment and other, net', 'COMPREHENSIVE INCOME'],
['Document Type', 'Document Quarterly Report', 'Document Period End Date', 'Document Fiscal Year Focus', 'Document Transition Report', 'Entity File Number', 'Entity Registrant Name', 'Entity Incorporation, State or Country Code', 'Entity Tax Identification Number', 'Entity Address, Address Line One', 'Entity Address, City or Town', 'Entity Address, State or Province', 'Entity Address, Postal Zip Code', 'City Area Code', 'Local Phone Number', 'Title of 12(b) Security', 'Trading Symbol', 'Security Exchange Name', 'Entity Current Reporting Status', 'Entity Interactive Data Current', 'Entity Filer Category', 'Entity Small Business', 'Entity Emerging Growth Company', 'Entity Shell Company', 'Entity Common Stock, Shares Outstanding', 'Amendment Flag', 'Document Fiscal Period Focus', 'Entity Central Index Key', 'Current Fiscal Year End Date'],
['Document Type', 'Document Quarterly Report', 'Document Period End Date', 'Document Fiscal Year Focus', 'Document Transition Report', 'Entity File Number', 'Entity Registrant Name', 'Entity Incorporation, State or Country Code', 'Entity Tax Identification Number', 'Entity Address, Address Line One', 'Entity Address, City or Town', 'Entity Address, State or Province', 'Entity Address, Postal Zip Code', 'City Area Code', 'Local Phone Number', 'Title of 12(b) Security', 'Trading Symbol', 'Security Exchange Name', 'Entity Current Reporting Status', 'Entity Interactive Data Current', 'Entity Filer Category', 'Entity Small Business', 'Entity Emerging Growth Company', 'Entity Shell Company', 'Entity Common Stock, Shares Outstanding', 'Amendment Flag', 'Document Fiscal Period Focus', 'Entity Central Index Key', 'Current Fiscal Year End Date']
]

# merged_list = merge_lists_unique(lists)
# print(merged_list)  # Output: ['A', 'B', 'C', 'D', 'E', 'F']


In [10]:
# Function to concatenate selected DataFrames into a master DataFrame
def concatenate_selected_dfs(selected_filings):
    # print(selected_filings)
    # {0: 0, 1: 0, 2: 0}
    # {0: 33, 1: 0, 2: 0} # location and selection

    statement_datas = [] # list of dfs 
    for filing_index, selected_index in selected_filings.items():
        statement_data = filings.iloc[filing_index]['df_statements'].iloc[selected_index]['df_table']
        # statement_name = filings.iloc[filing_index]['df_statements'].iloc[selected_index]['Internal Name']
        statement_datas.append(statement_data)

    listoflistof_concepts = []
    for statement_data in statement_datas:

        # this is really for each column in master, collect the concepts and the data
        concepts = statement_data[statement_data.columns[0]]
        # facts = statement_data[statement_data.columns[1]]

        listoflistof_concepts.append(list(concepts))

        # value_column = f'Value_{statement_name.split()[1]}'  # Extracting the statement name part (A, B, C, etc.)
        # statement_data.rename(columns={'Value': value_column}, inplace=True)  # Rename 'Value' column
        # statement_data['primaryDocument'] = filings.iloc[filing_index]['primaryDocument']  # Add Filing Name as a column

    master_concepts = merge_lists_unique(listoflistof_concepts)
    # print(master_concepts)


    # now that we have all the concepts, we need to iterate through each table, and attach each fact to the list 
    master_df = pd.DataFrame({"Concept":master_concepts})  # Initialize an empty DataFrame

    # print(master_concepts)

    # now that concepts have been added, we can just add the statement data for each statement datas 
    # A = statement_datas[0]
    # B = statement_datas[1]

    for statement_data in statement_datas:
        pass
        # master_df = pd.concat([master_df, statement_data], ignore_index=True)
        master_df = master_df.merge(statement_data, on='Concept', how='left')

    return master_df

# display(concatenate_selected_dfs({0: 0, 1: 0, 2: 0}))

<h1>Layout and Callbacks</h1>

In [11]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Function to generate radio buttons and detail display for each filing
def generate_filing_layout(filing_index, filing):
    filing_id = f'filing-{filing_index}'
    return html.Div([
        html.H2(f"{filing['reportDate']} {filing['form']}"),
        # html.Div(f"{filing['InteractiveURL']}"),
        # html.Div(f"{filing['ExcelURL']}"),
        dcc.RadioItems(
            id=f'{filing_index}-statement-radio',
            options=[{'label': name, 'value': i} for i, name in filing['df_statements']['Internal Name'].items()],
            value=0,
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        ),
        html.Div(id=f'{filing_id}-statement-details'),
        html.Div(id=f'{filing_id}-datatable-container')
    ], style={'flex': '1', 'margin': '10px'})

# Generate the layout for all filings
app.layout = html.Div([
    html.H1("Filings DataFrame"),
    html.Div(
        [generate_filing_layout(index, filing) for index, filing in filings.iterrows()],
        style={'display': 'flex', 'flex-wrap': 'wrap'}
    ),
    html.H2("Master DataFrame"),
    html.Div(id='master-dataframe')
])

# Callbacks to update the statement details for each filing
@app.callback(
    Output('master-dataframe', 'children'),
    [Input(f'{i}-statement-radio', 'value') for i, filing in filings.iterrows()],
    prevent_initial_call=True
)
def display_master_dataframe(*selected_indices):
    selected_filings = {i: idx for i, idx in enumerate(selected_indices)}
    master_df = concatenate_selected_dfs(selected_filings)

    master_df.to_csv("output.csv")
    return dash_table.DataTable(
        id='master-data-table',
        columns=[{"name": col, "id": col} for col in master_df.columns],
        data=master_df.to_dict('records'),
        style_table={'overflowX': 'scroll'},
    )

if __name__ == '__main__':
    app.run_server(debug=True)
